## 1. 문서 vector DB화

bge-m3 모델과 faiss를 활용하여 train 데이터를 vectorDB화 합니다.

In [ ]:
!pip install -U FlagEmbedding
!pip install faiss-gpu

  Using cached FlagEmbedding-1.2.11.tar.gz (147 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.4.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.2-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-

In [ ]:
import os
import faiss
import datasets
import numpy as np
from tqdm import tqdm
from FlagEmbedding import FlagModel
import json
import pandas as pd

In [ ]:
df = pd.read_csv('data/train.csv')

pdf parser 텍스트의 임베딩을 vectorDB에 저장하는 함수

In [ ]:
def generate_index(model: FlagModel, corpus: datasets.Dataset, max_passage_length: int=512, batch_size: int=256):
    corpus_embeddings = model.encode_corpus(corpus["content"], batch_size=batch_size, max_length=max_passage_length)
    dim = corpus_embeddings.shape[-1]

    faiss_index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
    corpus_embeddings = corpus_embeddings.astype(np.float32)
    faiss_index.train(corpus_embeddings)
    faiss_index.add(corpus_embeddings)
    return faiss_index, list(corpus["id"])

def save_result(index: faiss.Index, docid: list, index_save_dir: str):
    docid_save_path = os.path.join(index_save_dir, 'docid')
    index_save_path = os.path.join(index_save_dir, 'index')
    with open(docid_save_path, 'w', encoding='utf-8') as f:
        for _id in docid:
            f.write(str(_id) + '\n')
    faiss.write_index(index, index_save_path)

실행 코드

In [ ]:
embedding_model = FlagModel(
        'BAAI/bge-m3',
        pooling_method='cls',
        normalize_embeddings=True,
        use_fp16=True
    )

os.makedirs('vectorDB')
corpus_list = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Vetorizing"):
    corpus_list.append({'id': row['fname'], 'content': row['dialogue']} )

corpus = datasets.Dataset.from_list(corpus_list)

index, docid = generate_index(
            model=embedding_model,
            corpus=corpus,
            max_passage_length=8192,
            batch_size=4
        )

index_save_dir = os.path.join('vectorDB', 'train')

os.makedirs(index_save_dir)

save_result(index, docid, index_save_dir)

Inference Embeddings: 100%|██████████| 3115/3115 [01:19<00:00, 39.05it/s]


## 2. 문서 검색

In [ ]:
!conda install -c conda-forge nmslib -y
!conda install -c conda-forge openjdk -y
!conda install -c conda-forge openjdk=11 -y
!pip install pyserini==0.22.1
!pip3 install kiwipiepy

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.9.0
  latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.7.1



## Package Plan ##

  environment location: /root/.conda

  added / updated specs:
    - nmslib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libblas-3.9.0              |       8_openblas          11 KB  conda-forge
    libcblas-3.9.0             |       8_openblas          11 KB  conda-forge
    libgfortran-ng-7.5.0       |      h14aa051_20          23 KB  conda-forge
    libgfortran4-7.5.0         |      h14aa051_20         1.2 MB  conda-forge
    liblapack-3.9.0            |       8_openblas          11 KB  conda-forge
    libopenblas-0.3.12         |pthreads_hb3c22a3_1   

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from FlagEmbedding import BGEM3FlagModel
import torch
from pyserini.search.faiss import FaissSearcher, AutoQueryEncoder
import json

FaissSearcher를 생성하여 효율적인 유사도 기반 검색을 가능하게 합니다.

In [ ]:
device = torch.device("cuda")

query_encoder = AutoQueryEncoder(
        encoder_dir='BAAI/bge-m3',
        device=device,
        pooling='cls',
        l2_norm=True
    )

searchers = FaissSearcher(
        index_dir='vectorDB/train',
        query_encoder=query_encoder
    )


In [ ]:
test_df = pd.read_csv('data/test.csv')
train_df = pd.read_csv('data/train.csv')

In [ ]:
train_df[train_df['fname'] == 'train_0']

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기


In [ ]:
for _, row in tqdm(test_df.iterrows(), total=len(df), desc="Co"):
    df.at[_,'retrievers'] = searchers.search(
        query=row['dialogue'],
        k=1,
    )[0]

In [ ]:
test_df['retrievers'] = ''
for _, row in tqdm(test_df.iterrows(), total=len(df), desc="Searching Docs"):
    temp = searchers.search(
        query=row['dialogue'],
        k=1,
    )[0]
    test_df.at[_,'retrievers'] = train_df[train_df['fname'] == temp.docid]

Searching Docs: 100%|██████████| 499/499 [00:10<00:00, 46.02it/s]


In [ ]:
test_df['dialogue'][0]

'#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. \n#Person2#: 네, 실장님...\n#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?\n#Person2#: 네, 실장님. 시작하셔도 됩니다.\n#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.\n#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?\n#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.\n#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.\n#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?\n#Person2#: 이것은 내부와 외부 통신에 적용됩니다.\n#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.\n#Person2#: 그게 다신가요?\n#Person1#: 네. 이 메모를 오후 4시 전에 모든 직원에게 타이핑하여 배포해 주세요.'

In [ ]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "rtzr/ko-gemma-2-9b-it"

quantization_config_4bit = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config_4bit,
    low_cpu_mem_usage=True,
)

model.eval()

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<end_of_turn>")
]


def make_answer(instruction):
    messages = [
        {"role": "user", "content": f"{instruction}"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
    )

    print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))
    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)


Loading checkpoint shards: 100%|██████████| 10/10 [00:04<00:00,  2.27it/s]


In [ ]:
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Making Answer"):
        q = f"""
                다음 정보를 바탕으로 요구사항에 답하세요:

                {row['dialogue']}


                요구사항: 요약해주세요.

                주어진 요구사항에만 답변하세요. 예시를 참고해서 답변하세요.

                예시:

                정보: {row['retrievers']['dialogue'].values[0]}

                요약: {row['retrievers']['summary'].values[0]}

                답변:
                """

        answer = make_answer(q)

        test_df.at[_,'answer'] = answer

Making Answer:   0%|          | 1/499 [00:06<52:05,  6.28s/it]

더슨 씨는 모든 직원에게 즉시 메시지 사용을 금지하는 새로운 정책을 발표한다. 이 정책은 내부 및 외부 통신 모두에 적용되며, 위반 시 경고, 직무정지, 해고가 따른다. 직원들은 질문이 있을 경우 부서장에게 문의해야 한다. 





Making Answer:   0%|          | 2/499 [00:12<51:53,  6.26s/it]

#Person1#과 #Person2#는 출퇴근길 교통체증과 오염 문제에 대해  더 나은 해결책을 찾고자 노력합니다. 대중교통, 자전거 이용 등 다양한 방법을 고려해봤지만, 결국에는 재택근무가 가장 효과적인 방법이라는 결론을 내립니다. 





Making Answer:   1%|          | 3/499 [00:18<49:18,  5.96s/it]

...

Person1은 빨래를 한 적 없는 Person2에게 세탁기 사용법을 가르친다. Person2는 처음에는 막막하게 느꼈지만, 세탁 과정을 배우고 이해하며 빨래가 쉬운 일이라는 것을 깨달았다.  



Making Answer: 100%|█████████▉| 498/499 [46:16<00:05,  5.87s/it]

매튜와 스티브는 서로 오랜만에 만나 얘기합니다. 매튜가 집을 구하는 고민을 하고 있는데, 스티브는 이웃의 아파트를 소개해주겠다고 합니다. 매튜는  동네에 다시 이사갈 수 있다면 좋겠다고 말하며, 스티브는 이웃에게 문의해 보겠다고 약속합니다. 





Making Answer: 100%|██████████| 499/499 [46:24<00:00,  5.58s/it]

프랭크가 승진했음을 알리고 친구들과 파티를 열 계획을 Shared합니다. 프랭크는 승진으로 인해 새로운 집을 살 수 있게 되었고, 파티는 토요일 저녁에 열릴 예정입니다. 벳시는 파티에 참석할 것을 약속하고, 프랭크의 승진을 진심으로 축하하며 계속해서 긍정적인 면에 집중하자고 조언합니다.  





In [ ]:
import pandas as pd
submit_df = pd.read_csv('data/sample_submission.csv')

# 생성된 답변을 제출 DataFrame에 추가
submit_df['summary'] = [row['answer'] for _, row in test_df.iterrows()]
# submit_df['summary'] = submit_df['Answer'].fillna("요약문입니다.")

# 결과를 CSV 파일로 저장
submit_df.to_csv("few_shot_llm_submission.csv", encoding='UTF-8-sig', index=False)